# Energy/Eigenvalue Distribution

In [15]:
%matplotlib inline
%cd /home/aditya/Documents/SOMBL
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scripts.plot_utils as putils
sns.set()
sns.set_style("whitegrid")

/home/aditya/Documents/SOMBL


In [16]:
%config InlineBackend.figure_format = 'svg'

In [17]:
kwargs = {
    "size": 10,
    "coupling": 0.,
    "disorder": 15,
    "hopup": 1.0,
    "hopdn": 1.0,
    "runs": 100,
    "nospin": False,
    "binnum": None,
    "alpha": None,
    "beta": None,
    "batchsize": 10,
    "batch": None
}

prefix = "data/mbl"

In [20]:
energy_core_min = -8
energy_core_max = 8

In [31]:
for batch in range(kwargs["batchsize"]):
    kwargs["batch"] = batch
    params = putils.SystemParams(**kwargs)
    filename = putils.getFilename(params, endname="_eigvals.dat", prefix=prefix)
    # print(filename)
    eigvals = np.loadtxt(filename)
    fig, ax = plt.subplots(figsize=(5, 5))
    palette = sns.color_palette("Blues")
    kdecolor = palette[3]
    linecolor = palette[5]
    # print(color)
    for i in range(params.batchsize):
        sns.kdeplot(eigvals[i, :], x="Energy", label=f"{i}", fill=True, alpha=1/params.batchsize,
                    color=kdecolor, ax=ax)
        sns.rugplot(eigvals[i, :], x="Energy", label=f"{i}", color=kdecolor, alpha=1/params.batchsize, ax=ax)
    ax.set_xlabel("Energy")
    ax.set_ylabel("Density")
    ax.axvline(energy_core_min, 0, 1, linestyle="--", color=linecolor)
    ax.axvline(energy_core_max, 0, 1, linestyle="--", color=linecolor)
    fig.tight_layout()
    savefilename = putils.getFilename(params, endname="_eigvals_density", prefix="mbl")
    fig.savefig("plots/PDFs/" + savefilename + ".pdf")
    fig.savefig("plots/PNGs/" + savefilename + ".png")
    plt.close(fig)